In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install texthero
import pandas as pd
import numpy as np
import texthero as hero
from texthero import preprocessing
from sklearn.metrics import accuracy_score, confusion_matrix, plot_confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier 
from imblearn.over_sampling import SMOTE

     |████████████████████████████████| 245kB 13.3MB/s 
     |████████████████████████████████| 1.4MB 25.3MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp36-none-any.whl size=1434677 sha256=1d6dbb373ae79ee719dc8f3fa9fd68578c30a16649b70e0dd604cbd16f1fe66e
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning:

The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.



In [3]:
df = pd.read_csv('/content/gdrive/My Drive/Deakin energy/processed_dataset_IG.csv')
df

,Unnamed: 0,ActionTaken,Address,CauseCommunity,CauseEnvironment,CausePre,CauseTechnical,CauseWorkP,ContactType,CorrectProtection,EventDescription,FailedAssets,FailedExplosion,FailedOilFilled,FailedOtherAssets,IncidentCause,IncidentConsequence,IncidentFireFFactorReportable,IncidentFireSeverity,IncidentLocationType,IncidentType,Lat,Long,MadeSafe,NetworkType,Status,Voltage,WeatherStation,Postcode,Locality,Category
0,0,Crew isolated supply and undertook repairs,"PARA PARK, 1490 HENDY MAIN ROAD, PARAPARAP VIC...",Vehicle,Lightning,Unlikely,Earth fault,Not followed,Phase to earth,2.0,A nearby customer reported sparking of electri...,Conductor (ABC),0,0,0,HVABC cable faulted midspan,Grassfire,1.0,Small: 10 - 1000 sq.m,Roadway,Infrastructure (network-based),-38.288982,144.191502,1,Powercor,Report,22kV AC,Avalon Airport,3240.0,Paraparap,OH Cable
1,1,Crew undertook repairs,"1154-1198 CHRISTIES ROAD, RAVENHALL VIC 3023",Working too close to underground cables,Lightning,Unlikely,Leakage,Not followed,Phase to earth,2.0,A contractor reported that he had contacted an...,No failed asset,0,0,0,Contractor contacted earthing conductor,No Go Zone (contact),1.0,Negligible: no ground fire,Roadway,Infrastructure (network-based),-37.767377,144.738509,1,Powercor,Report,Earthing cable,Laverton Raaf,3023.0,Ravenhall,Dug up
2,2,Crew isolated supply and undertook repairs,"326 DUNBARS ROAD, PETERBOROUGH VIC 3270",Vehicle,Lightning,Unlikely,Corrosion,Not followed,Phase to earth,1.0,A field crew attending an outage found that a ...,Conductor (bare),0,0,0,Conductor broke due to rust,Serious risk to public safety (e.g. live condu...,1.0,Negligible: no ground fire,Agricultural,Infrastructure (network-based),-38.586647,142.901137,1,Powercor,Report,22kV AC,Warrnambool Airport Ndb,3270.0,Peterborough,Connection
3,3,Crew undertook repairs,"41-49 BANK STREET, SOUTH MELBOURNE VIC 3205",Vandalism,Lightning,Unlikely,Leakage,Not followed,Phase to earth,1.0,Interfere and vandalism in substation. Unknown...,Indoor,0,2,0,Vandalism,Loss of supply;Damage to network assets,1.0,Negligible: no ground fire,Roadway,Infrastructure (network-based),-37.832352,144.968452,1,CitiPower,Report,22kV AC,Essendon Airport,3205.0,South Melbourne,Other
4,4,Crew isolated supply and undertook repairs,"199 WILSONS ROAD, WHITTINGTON VIC 3219",Vehicle,Lightning,Unlikely,Leakage,Not followed,Phase to earth,2.0,A nearby customer reported that a high load ha...,Service conductor,0,0,0,Unknown high load contacted LV service cable,No Go Zone (contact);Damage to network assets,1.0,Negligible: no ground fire,Roadway,Infrastructure (network-based),-38.177436,144.390480,1,Powercor,Report,Low voltage AC (<1kV),Avalon Airport,3219.0,Whittington,Vehicle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6438,6499,The crew replaced a 3 phase FOLCB on the pole,"301 ST KILDA STREET, BRIGHTON VIC 3186",Vehicle,Lightning,Unlikely,Earth fault,Not followed,Phase to earth,2.0,A report came in to UE of a FMB sparking at 31...,Connection box,0,0,0,The likely cause was determined by EIO to be a...,Plant/equipment,1.0,Negligible: no ground fire,Residential,Infrastructure (network-based),-37.905307,144.988061,1,United Energy Distribution,Report,Low voltage AC (<1kV),Moorabbin Airport,3186.0,Brighton,Connection
6439,6500,Conductor repaired at point of contact.,"48 Banks Rd, Mount Wallace VIC 3342, Australia",Working too close to overhead lines,Lightning,Unlikely,Leakage,Not followed,Phase to earth,2.0,Report received from a resident to advise that...,Conductor (bare),2,2,2,Crane struck line,No Go Zone (contact);Damage to network assets,1.0,Negligible: no ground fire,Agricultural,Infrastructure (network-based),-37.707726,144.199888,1,Powercor,Report,12.7kV AC (SWER),Avalon Airport,3342.0,Mount Wallace,Vehicle
6440,6501,On arrival crew isolated supply and arranged p...,"196 VALLANCE ROAD, OUYEN VIC 3490",Vehicle,Lightning,Unlikely,Leakage,Not followe

In [4]:
def data(df):
    features = df['EventDescription'] +' ' + df['Address'] + ' '+ df['IncidentCause'] +' ' + df['ActionTaken'] +\
    ' '+ df['FailedAssets'] +' ' + df['Locality']  + ' ' + df['IncidentConsequence'] + ' ' + \
    df['CauseTechnical'] + ' ' + df['CauseTechnical']
    target = df['Category']
    return features, target

features, target = data(df)
features

0       A nearby customer reported sparking of electri...
1       A contractor reported that he had contacted an...
2       A field crew attending an outage found that a ...
3       Interfere and vandalism in substation. Unknown...
4       A nearby customer reported that a high load ha...
                              ...                        
6438    A report came in to UE of a FMB sparking at 31...
6439    Report received from a resident to advise that...
6440    Report received of pole fire at incident locat...
6441    A customer called to report a tractor had hit ...
6442    concrete electrical cover outside of front of ...
Length: 6443, dtype: object

In [5]:
def preprocess_text():
    # cleaning steps
    cleaning_pipeline = [
        preprocessing.fillna,
        preprocessing.lowercase,
        preprocessing.remove_whitespace,
        preprocessing.remove_punctuation,
        preprocessing.remove_urls,
        preprocessing.remove_brackets,
        preprocessing.remove_stopwords,
        preprocessing.remove_digits,
        preprocessing.remove_angle_brackets,
        preprocessing.remove_curly_brackets,
        preprocessing.stem
    ]

    # apply pipeline to text
    clean_text = features.pipe(hero.clean, cleaning_pipeline)

    return clean_text


clean_text = preprocess_text()
clean_text

0       nearbi custom report spark electr line locat a...
1       contractor report contact earth cabl excav tre...
2       field crew attend outag found 22kv conductor b...
3       interfer vandal substat unknown third parti us...
4       nearbi custom report high load pull wire locat...
                              ...                        
6438    report came ue fmb spark st kilda st brighton ...
6439    report receiv resid advis crane made contact p...
6440    report receiv pole fire incid locat arriv crew...
6441    custom call report tractor hit wire brought gr...
6442    concret electr cover outsid front hous gave cl...
Length: 6443, dtype: object

In [33]:
x_train, x_test, y_train, y_test = train_test_split(clean_text, target, random_state=0, test_size=0.25, shuffle=True)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((4832,), (1611,), (4832,), (1611,))

In [34]:
def tfidf():
    vectorizer = TfidfVectorizer(analyzer='word', max_features=3000) #best at max features = 3000
    vectorizer.fit(clean_text)
    train_tfidf = vectorizer.transform(x_train)
    test_tfidf = vectorizer.transform(x_test)
    return train_tfidf, test_tfidf

train_tfidf, test_tfidf = tfidf()
train_tfidf.shape, test_tfidf.shape

((4832, 3000), (1611, 3000))

In [35]:
import numpy as np
oversample = SMOTE(random_state=0,n_jobs=-1,k_neighbors=5)
train_tfidf, y_train = oversample.fit_resample(train_tfidf, y_train)
print(f'Shape: {train_tfidf.shape}')

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.



Shape: (14820, 3000)


In [36]:
dt = DecisionTreeClassifier(random_state=1)
dt.fit(train_tfidf ,y_train)
print(accuracy_score(y_test, dt.predict(test_tfidf)))
print(classification_report(y_test, dt.predict(test_tfidf)))

0.7318435754189944
              precision    recall  f1-score   support

    AF Other       0.39      0.39      0.39        93
      Animal       0.75      0.69      0.72        67
   Conductor       0.25      0.44      0.32        34
  Connection       0.70      0.71      0.70       210
    Crossarm       0.82      0.78      0.80       130
      Dug up       0.76      0.75      0.75        92
        Fuse       0.78      0.84      0.81       107
Installation       0.25      0.25      0.25         8
   Lightning       0.94      0.78      0.85        41
    OH Cable       0.64      0.63      0.64        60
       Other       0.82      0.80      0.81       325
        Pole       0.51      0.52      0.51        83
       Trees       0.89      0.90      0.90       134
    UG Cable       0.25      0.17      0.20         6
     Vehicle       0.82      0.79      0.81       221

    accuracy                           0.73      1611
   macro avg       0.64      0.63      0.63      1611
weighte

In [19]:
#Conclusion
#Accuracy decreased with Information Gain